In [2]:
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [4]:
h2o.init()
h2o.remove_all()

dataset = 'Datasets/colon-data.txt'
dataclass = 'Datasets/colon-class.txt'

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Australia/Sydney
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.6
H2O cluster version age:,1 month and 20 days
H2O cluster name:,H2O_from_python_andyg_84jkj8
H2O cluster total nodes:,1
H2O cluster free memory:,3.523 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [5]:
# Data preparation and manipulation
colon_data = h2o.import_file(path=dataset)
colon_class = h2o.import_file(path=dataclass, header=-1, sep='\t')

colon_class_pd = colon_class.as_data_frame(use_pandas=True)
colon_class_pd = colon_class_pd.transpose()
colon_class_pd.columns = ['class']
colon_class = h2o.H2OFrame(colon_class_pd)

colon_data_pd = colon_data.as_data_frame(use_pandas=True)
colon_data_pd = colon_data_pd.transpose()
colon_data_pd.columns = colon_data_pd.iloc[0]
colon_data_pd = colon_data_pd.drop(labels='categories', axis=0)
colon_data = h2o.H2OFrame(colon_data_pd)

colon = colon_data.cbind(colon_class)
colon_pd = colon.as_data_frame(use_pandas=True)
colon_pd.sample(frac=1).reset_index(drop=True)
colon = h2o.H2OFrame(colon_pd)

label = 'class'
features = colon.names
features.remove(label)

train, valid = colon.split_frame([0.80])
train[label] = train[label].asfactor()
valid[label] = valid[label].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


C:\Users\andyg\Anaconda3\envs\tensorflow\lib\site-packages\h2o\utils\shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
# Model properties and training
model = H2ODeepLearningEstimator(activation = "rectifier_with_dropout", hidden = [190,63,21,7], epochs = 50, input_dropout_ratio = 0.1)
model.train(x = features, y = label, training_frame = train, validation_frame = valid)

deeplearning Model Build progress: |████████████████████████████████████████| 100%


In [7]:
# Model predictions and validation
predictions = model.predict(valid)
print(predictions)
print(model.confusion_matrix(valid=True))

deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,Normal,Tumor
Normal,0.990158,0.00984208
Normal,0.879661,0.120339
Tumor,0.000897613,0.999102
Tumor,0.000612421,0.999388
Tumor,0.00178321,0.998217
Tumor,0.00202495,0.997975
Tumor,0.000518104,0.999482
Tumor,0.00363839,0.996362
Tumor,9.02031e-05,0.99991
Tumor,0.000850963,0.999149



Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9809016563663026: 


,Normal,Tumor,Error,Rate
Normal,2.0,0.0,0.0,(0.0/2.0)
Tumor,0.0,13.0,0.0,(0.0/13.0)
Total,2.0,13.0,0.0,(0.0/15.0)
